In [1]:
map_dir = '/global/cscratch1/sd/hongbo/lens_rot_bias/Maps/'
for i in range(10):
    for j in range(10):
        if i!=j:
            map_list = []
            # map_list.append(map_dir+'CMBRot_fullsky_alm_1e-07_00%s.fits'%i)
            # map_list.append(map_dir+'CMBRot_fullsky_alm_1e-07_00%s.fits'%j)
            # map_list.append(map_dir+'CMBRot_fullsky_alm_1e-07_00%s.fits'%(i+10))
            # map_list.append(map_dir+'CMBRot_fullsky_alm_1e-07_00%s.fits'%(j+10))
            # map_list.append('CMBRot_fullsky_alm_1e-07_00%s.fits'%i)
            # map_list.append('CMBRot_fullsky_alm_1e-07_00%s.fits'%j)
            # map_list.append('CMBRot_fullsky_alm_1e-07_00%s.fits'%(i+10))
            # map_list.append('CMBRot_fullsky_alm_1e-07_00%s.fits'%(j+10))
            # print(map_list)

In [2]:
import sys, os, time
import numpy as np
import param as p

In [9]:
# add_noise = False
# qos = 'regular'
# time = '09:00:00'
# OMP_NUM_THREADS = 32
# n_tasks_per_node = int(nrecon_per_job * OMP_NUM_THREADS)

job_path = p.repodir + 'N1aaJobs/'
src_path = p.repodir + 'src'

In [7]:
# arguments list for all processes
A_cb = 1e-07
args_list = []
# recon sim jobs
recon_sim_job_nums = [i for i in range(20)]
sim_nums = [i for i in range(10)]

for experiment, values  in p.exps_config.items():
    for groups, moment in p.moments.items():
        for i in sim_nums:
            for j in sim_nums:
                if i!=j:
                    args_dict = {}
                    args_dict['cmb1'] = p.repodir + f'Maps/CMBRot_fullsky_alm_{A_cb}_{i:03d}.fits'
                    args_dict['cmb2'] = p.repodir + f'Maps/CMBRot_fullsky_alm_{A_cb}_{j:03d}.fits'
                    args_dict['cmb3'] = p.repodir + f'Maps/CMBRot_fullsky_alm_{A_cb}_{i+10:03d}.fits'
                    args_dict['cmb4'] = p.repodir + f'Maps/CMBRot_fullsky_alm_{A_cb}_{j+10:03d}.fits'
                    args_dict['experiment'] = experiment
                    args_dict['nlev_t'] = values['nlev_t']
                    args_dict['nlev_p'] = values['nlev_p']
                    args_dict['beam_arcmin'] = values['beam_arcmin']
                    args_dict['ellmin' ]= moment['ellmin']
                    args_dict['ellmax' ]= moment['ellmax']
                    args_dict['delta_L'] = moment['delta_L']
                    args_list.append(args_dict)

recon_param_chuncks = np.array_split(args_list, len(recon_sim_job_nums))

In [10]:
for i, recon_job in enumerate(recon_sim_job_nums):
    f = open('%s/reconJob.sh.%d' %(job_path, recon_job), 'w')
    
    f.write('#!/bin/bash\n')
    # f.write('#SBATCH --qos %s\n' %qos)
    # f.write('#SBATCH --constraint=haswell\n')
    # f.write('#SBATCH -N 1\n')
    # f.write('#SBATCH -t %s\n' %time)
    # f.write('#SBATCH --ntasks-per-node=%s\n' %n_tasks_per_node)
    # f.write('#SBATCH --license=SCRATCH\n')
    # f.write('#SBATCH --output=%sslurmjob.log.%d\n' %(job_path, recon_job))
    # f.write('\n')
    f.write('cd %s\n\n' %src_path)
    for recon_param in recon_param_chuncks[i]:
        f.write('python reconN1aa.py --cmb1 \'%s\' --cmb2 \'%s\' --cmb3 \'%s\' --cmb4 \'%s\' --experiment \'%s\' --nlev_t %s --beam_arcmin %s --ellmin %s --ellmax %s --delta_L %s & sleep 1\n' %(recon_param['cmb1'], recon_param['cmb2'], recon_param['cmb3'], recon_param['cmb4'], recon_param['experiment'], recon_param['nlev_t'], recon_param['beam_arcmin'], recon_param['ellmin'], recon_param['ellmax'], recon_param['delta_L']))
    f.write('wait\n')
    f.close()